# Spatial Autocorrelation with PySAL

## Global Moran's I exercise

Using this Jupyter notebook, you will code the same process that we used in GeoDa to run the Univariate Global Moran's I to measure overall level of Spatial Autocorrelation.

Adapted from https://pysal.readthedocs.org/en/latest/users/tutorials/autocorrelation.html#moran-s-i
Information also available on pg 35 of the PySAL_Documentation.pdf included with the SDS Bootcamp materials.

Note: to run a cell, click in the cell, then go to top menu and select Cell > Run Cells.
You can also click in the cell, and hold down Ctrl and Enter together. 


### Task #1: Explore and Visualize Dataset

#### First, import the needed Python components - all scripting in Python begins with import

In [ ]:
# PySAL and Numpy the only ones needed to actually run the spatial autocorrelation analysis
import pysal
import numpy as np

#Folium is used to create some map visualizations
import folium

# These others are to handle, query, and plot data
import os
import pandas as pd
import geopandas as gpd
import json
import simpledbf
%matplotlib inline
import matplotlib.pyplot as plt

# This message below will print after the commands above are successfully completed
print ('All requested Python libraries imported successfully')

In [ ]:
# Explore the data table using pandas 

# Use simpledbf to read the attribute table (dbf file)
dbf = simpledbf.Dbf5('/home/ubuntu/Documents/Counties/cnty_Lyme_disease.dbf')

# Convert dbf file to a pandas data frame
df = dbf.to_dataframe()

# Create a list of the columns and print the column names 
columns = list(df)
print(columns)

In [ ]:
# Extract the values for 2005 
extracted = df.loc[:,["County","2005"]]

# Print the extracted values for 2005
extracted

In [ ]:
# Order the counties by highest to lowest count for 2005 
# http://pandas.pydata.org/pandas-docs/version/0.18.1/generated/pandas.DataFrame.sort.html

# ascending is the parameter used to define the sorting order
# a value of 1 for ascending means true and orders in ascending order
# a value of 0 for ascending means false and orders in descending order instead
result = extracted.sort_values(by=["2005", "County"], ascending=[0, 1])

# Print the sorted values for 2005
result

In [ ]:
# We can also query values for specific counties from our extracted dataframe for 2005
print (extracted[extracted.County == 'Alameda'])

In [ ]:
# To use Folium for map visualizations, we convert the shapefile to GeoJSON

# We can use GeoPandas for this conversion
# First, read in the shapefile to geopandas
shapefile = gpd.read_file('/home/ubuntu/Documents/Counties/viz/cnty_Lyme_disease_WGS84.shp').set_index('NAME_PCASE')

# Next, save the file out to GeoJSON
with open('/home/ubuntu/Documents/Counties/viz/cnty_Lyme_disease_WGS84.geojson', 'w') as f:
    f.write(shapefile.to_json())

# This message below will print after the commands above are successfully completed
print ('Successfully converted shapefile to geojson')

In [ ]:
# Explore the data visually - create a basic visualization map of California counties

# Read in the GeoJSON file created in previous step
counties = '/home/ubuntu/Documents/Counties/viz/cnty_Lyme_disease_WGS84.geojson'
geo_json_data = json.load(open(counties))

# Create an empty Folium map and fill it with the GeoJSON data
m = folium.Map([37, -122], zoom_start=6)
folium.GeoJson(geo_json_data).add_to(m)

# Save the map to html
m.save('/home/ubuntu/Documents/Counties/viz/basic_countymap.html')

# Print the map to screen
m

In [ ]:
# Explore the data visually - create a choropleth map of the 2005 counts of lyme disease 

# Use simpledbf to read the attribute table (dbf file)
dbf = simpledbf.Dbf5('/home/ubuntu/Documents/Counties/cnty_Lyme_disease.dbf')

# Convert dbf file to a pandas data frame
df = dbf.to_dataframe()

# Extract the values for 2005 
extracted = df.loc[:,["County","2005"]]

# Create an empty Folium map
m = folium.Map([37, -122], zoom_start=6)

# Join the extracted 2005 values to the map
m.choropleth(
    geo_str=open(counties).read(),
    data=extracted,
    columns=['County', '2005'], 
    key_on='properties.County',
    fill_color='YlGn',
)

# Save the map as html
m.save(os.path.join("/home/ubuntu/Documents/Counties/viz/2005_countymap.html"))

# Print the map to screen
m

### Task #2: read data into PySAL and create new variables for analysis

#### To calculate Moran's I, we need to give it a list of neighbors. We can do this by reading in a spatial weights matrix.
Remember that PySAL likes the GAL file format, which we created in the GeoDa exercise. 

This file can be converted to an ArcGIS Spatial Weights Matrix (SWM) file, and vice versa.
PySAL can read the GAL file as follows: counties = pysal.open('path/to/file/called/file.gal').read()

#### Alternatively, and much easier, PySAL can read the neighbors directly from a shapefile

In [ ]:
# Instead of reading in the .gal file we created in GeoDa, we will ask PySAL to create one from the shapefile.
counties = pysal.queen_from_shapefile('/home/ubuntu/Documents/Counties/cnty_Lyme_disease.shp')

# This message below will print after the command above is successfully completed
print ('New weights file successfully created')

#### The queen_from_shapefile function has defined neighbors using the the queen weights criteria, which defines a location's neighbors as those areas with at least one shared corner

#### Other options include a rook weights matrix, in which neighbors need to share an entire border (i.e. a line of two connected vertices)

In [ ]:
# counties has some atributes associated with it
# For example, n which is equal to number of features in the data
print (counties.n)

In [ ]:
# Use the help function to find out what other attributes you can call
# You can clear/remove this output from your view when you are done
# Go to Cell > Current Outputs > Clear
help(counties)

In [ ]:
# Remember it is important to make sure that you have no "islands" in your dataset
# Which attribute of counties can you use to list islands?
counties.islands

In [ ]:
# Next, let's read in the dbf that contains data for the counties
# http://www.pysal.org/users/tutorials/fileio.html

table = pysal.open('/home/ubuntu/Documents/Counties/cnty_Lyme_disease.dbf')

# This message below will print after the command above is successfully completed
print ('DBF file successfully imported')

# Return the column headers from dbf
table.header

In [ ]:
# Next, specify which column contain the variable of interest
# Notice that we are using the array function from numpy, which we named np during our import
# This array will contain the data from the column called 2005

lymecases = np.array(table.by_col['2005'])

# This message below will print after the command above is successfully completed
print ('A variable called lymecases successfully created')

### Task #3: complete a single run of Global Moran's I to measure overall level of Spatial Autocorrelation 

In [ ]:
# Using the functions examples below, update the parameters to run Global Moran's I on the year 2005 data

# In the online tutorial, the function reads as follows: mi = pysal.Moran(y,w)
# In their example, y = array containing homicide rates and w = spatial weights variable for the neighbors

# Another example could be something like: mi = pysal.Moran(crimeindex, blockgroups)
# In this example, crimeindex is the array containing a crime index and blockgroups is the spatial weights variable 

# Hint: mi = pysal.Moran(y,w)
mi = pysal.Moran(lymecases, counties)

In [ ]:
# Again, examine the help to learn more about the outputs from your function
help(mi)

In [ ]:
# Which attribute of mi can you use to see the actual observed value of spatial autocorrelation in the dataset?
# Hint: mi.attributename
mi.I

In [ ]:
# Which attribute of mi can you use to see the expected value of spatial autocorrelation? 
# Hint: mi.attributename
mi.EI 

In [ ]:
# Which attribute of mi can you use to see the statistical significance of difference between I and EI?
# Remember the goal is p < 0.05
# Hint: mi.attributename
mi.p_norm 

### Task #4: check whether null hypothesis is rejected after a single run of Global Moran's I

In [ ]:
# Use your cheat sheet and the example below to write an If/Else statement to print a message about the p value
# If the p value is less than 0.05, then you can reject the null hypothesis that the lyme disease cases are randomly distributed

# useful online resources: 
# https://docs.python.org/2/tutorial/controlflow.html
# http://learnpythonthehardway.org/book/ex30.html

# Hint: If/Else Statement From the Presentation Slides
# if x < 2:
    # print ('Some Message.')
# else:
    # print ('A different message.')

if mi.p_norm < 0.05:
    print ('Based on one run, lyme disease cases do not appear to be randomly distributed. There may be spatial autocorrelation in the data')
else:
    print ('Based on one run, lyme disease cases appear to be randomly distributed.')

### Task #5: add permutations (multiple runs) of the random landscape

In [ ]:
# Create a new instance of the pysal.Moran function that includes a parameter value for permutations
# Name this new instance: mir

# For more info: https://pysal.readthedocs.org/en/latest/users/tutorials/autocorrelation.html#moran-s-i
# Hint: Examine the function mir and what parameters are allowed
# Hint: help(mir)

mir = pysal.Moran(lymecases, counties, permutations = 9999)

print ('Multiple permutations version of mi')

In [ ]:
# Which attribute of mir can you use to see the actual observed value of spatial autocorrelation in the dataset?
# Hint: help(mir)
# Hint: mir.attributename

print ('Observed value for I: ', mir.I)

In [ ]:
# Which attribute of mir can you use to see the expected value of spatial autocorrelation based on the randomized versions of the data (simulations)? 
# Hint: mir.attributename

print ('Expected value for I: ', mir.EI_sim)

In [ ]:
# Which attribute of mir can you use to see the statistical significance of difference between observed I and simulated I values?
# Remember the goal is p < 0.05
# Hint: mir.attributename
# Hint: the output variable with the p value has a slightly different name than before

print ('Calculated pseudo p value based on these permutations: ', mir.p_sim)

In [ ]:
# Does the pseudo p value support that there is statistical difference between the observed I and simulated I values?

# Maybe a visualization can help - plot the distribution of random permutations for Moran's I, similar to the one in GeoDa
# Compare the observed I of the dataset to the distribution of I for the random permutations

# Create an empty list to store values after each run 
permutation = []

# Calculate the Moran's I for 1000 random distributions of the dataset and store them in a list called permutation
for i in range(0, 1000):
    new_permute = np.random.choice(lymecases,size = len(lymecases), replace = False)
    permuted_moran = pysal.Moran(new_permute,counties)
    permutation.append(permuted_moran.I)

# Plot the histogram    
plt.hist(permutation)

# Add a verrtical line to display the actual value Moran's I of the dataset
plt.axvline(x= mi.I, color='y', linestyle='--', linewidth = 1)

### Task #6:  check whether null hypothesis is rejected after multiple runs of Global Moran's I

In [ ]:
# Again, Use your cheat sheet and the example below to write an If/Else statement to print a message about the p value
# If the p value is less than 0.05, then you can reject the null hypothesis that the lyme disease cases are randomly distributed

# Hint: the output variable with the p value has a slightly different name than before

if mir.p_sim < 0.05:
    print ('Based on multiple runs, lyme disease cases still do not appear to be randomly distributed. There may be spatial autocorrelation in the data')
else:
    print ('Based on multiple runs, lyme disease cases still appear to be randomly distributed.')


## Bonus Exercises for Global Moran's I exercise

### Bonus #1: examine the "z transformation" values for added robustness

In [ ]:
# Which attributes of mir show you the z transformation values?
# For more info: https://pysal.readthedocs.org/en/latest/users/tutorials/autocorrelation.html#moran-s-i
# Hint: help(mir) and look for output variables with "z" in the name

print ('Calculated z value based on these permutations: ', mir.z_sim)

print ('Calculated p value based on these permutations, using a z transformation: ', mir.p_z_sim)

### Bonus #2:  check whether null hypothesis is rejected based on "z transformation"

In [ ]:
# Again, use your cheat sheet and the example below to write an If/Else statement to print a message 
# this time, print a message about the z value

# based on a 95% confidence level, z-value between -1.96 and +1.96 does not reject null hypothesis
# this would mean that the data are randomly distributed through space

# Hint: the output variable with the p value has a slightly different name than before
# Hint: you will need to use an "or" in the if/else statement
# useful online resources: 
# https://docs.python.org/2/tutorial/controlflow.html
# http://learnpythonthehardway.org/book/ex30.html

if mir.z_sim > 1.96 or mir.z_sim < -1.96:
    print ('Based on the z transformation of p value, the null hypothesis that homicides are randomly distributed is rejected.')
else:
    print ('Based on the z transformation of p value, the null hypothesis that homicides are randomly distributed is not rejected.')


## Advanced Bonus Exercises for Global Moran's I exercise

### Advanced Bonus #1: use a loop to run Global Moran's I for other years of data (2006 to 2014)

In [ ]:
# Note: these advanced bonus exercises may be challenging for new Python programmers 
# Hint: check out Lists as an iterable on: https://wiki.python.org/moin/ForLoop
# Hint: create a list of the years to begin

yearlist = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']

yearlist



In [ ]:
# Note: these advanced bonus exercises are challenging for new Python programmers 
# Hint: check out Lists as an iterable on: https://wiki.python.org/moin/ForLoop 
# Hint: now use a loop to execute mir for each of the years in your list
# Hint: use 'print' to display your output and be sure to output mir.p_z_sim and mir.z_sim

for year in yearlist:
    lymecases = np.array(table.by_col[year])
    mir = pysal.Moran(lymecases, counties, permutations = 9999)
    print (year, mir.I, mir.EI_sim, mir.p_z_sim, mir.z_sim)
    

### Advanced Bonus #2: expand your loop to capture results from all years into pandas dataframe

In [ ]:
# Note: these advanced bonus exercises are challenging for new Python programmers 
# Hint: check out Lists as an iterable on: https://wiki.python.org/moin/ForLoop 
# Hint: check out Append on http://pandas.pydata.org/pandas-docs/stable/10min.html 
# Hint: create some empty lists and expand your loop to include a few appends to these empty lists
# Hint: use 'print' to display your output and be sure to output mir.p_z_sim and mir.z_sim

moranlist = []
emoranlist = []
pvaluelist = []
zvaluelist = []

for year in yearlist:
    lymecases = np.array(table.by_col[year])
    mir = pysal.Moran(lymecases, counties, permutations = 9999)
    moranlist.append(mir.I)
    emoranlist.append(mir.EI_sim)
    pvaluelist.append(mir.p_z_sim)
    zvaluelist.append(mir.z_sim)
    
df = pd.DataFrame({'Year' : yearlist, 'Actual Morans I' : moranlist, 'Expected Morans I': emoranlist, 'P_value' : pvaluelist, 'Z_value': zvaluelist})

cols1 = df.columns.tolist()
cols1.insert(0, cols1.pop(cols1.index('Year')))
df = df.ix[:, cols1]
df


### Advanced Bonus #3: query your pandas dataframe for years with significant Global Moran's I

In [ ]:
# Note: these advanced bonus exercises are challenging for new Python programmers 
# Hint: check out Boolean Indexing on http://pandas.pydata.org/pandas-docs/stable/10min.html 
# Hint: query for p values < 0.05 in the pandas dataframe

df[df.P_value < 0.05]



#### Final thoughts: Notice that none of our z values are negative which would indicate dispersion in the data.  Most years do not have a significant p value of the difference between the observed I and the expected value of I, thus indicating that the lyme disease cases appear to be randomly distributed.  Similarly, most years also do not have significant positive z values (which would indicate a global clustered pattern).

#### For the years that you do reject the null hypothesis (random distribution), what does this mean, as there are not many of them?

#### Does all this mean that there is no spatial autocorrelation in lyme disease cases in California?

## End of Global Moran's I exercise


### Other options to continue your exploration:
#### Move on to the next exercise for the Local Indicators of Spatial Autocorrelation exercise to explore local hot and cold spots (PySAL - Spatial Autocorrelation - Local Morans I - Start with this script.ipynb)

#### You can continue to explore pandas; can you limit the last dataframe again to only significant z values? (refer to Bonus #2) http://pandas.pydata.org/pandas-docs/stable/10min.html


In [ ]:
# Hint: check out Boolean Indexing on http://pandas.pydata.org/pandas-docs/stable/10min.html

finaldf = df[df.Z_value > 1.96]

finaldf



### Or just for fun: explore some more advanced visualizations below 

In [ ]:
# Explore the data visually - create a choropleth map of the total disease count from 2005-2014 (sum)

# Read in the GeoJSON file created earlier
counties = '/home/ubuntu/Documents/Counties/viz/cnty_Lyme_disease_WGS84.geojson'
geo_json_data = json.load(open(counties))

# Use simpledbf to read the attribute table (dbf file)
dbf = simpledbf.Dbf5('/home/ubuntu/Documents/Counties/cnty_Lyme_disease.dbf')

# Convert dbf file to a pandas data frame
df = dbf.to_dataframe()

# Compute the sum of number of diseases from 2005 to 2014
df["sum"] = df.loc[:,"2005":"2014"].sum(axis = 1)

# Extract the columns needed for the choropleth
extracted = df.loc[:,["County","sum"]]

# Create an empty Folium map
m = folium.Map([37, -122], zoom_start=6)

# Join the extracted values to the map
m.choropleth(
    geo_str=open(counties).read(),
    data=extracted,
    columns=['County', 'sum'],
    key_on='properties.County',
    fill_color='YlGn',
)

# Save the map as html
m.save(os.path.join("/home/ubuntu/Documents/Counties/viz/sum_countymap.html"))

# Print the map to screen
m

In [ ]:
# Explore the data visually - create a choropleth map of the range of disease counts from 2005-2014 (max-min)

# Read in the GeoJSON file created earlier
counties = '/home/ubuntu/Documents/Counties/viz/cnty_Lyme_disease_WGS84.geojson'
geo_json_data = json.load(open(counties))

# Use simpledbf to read the attribute table (dbf file)
dbf = simpledbf.Dbf5('/home/ubuntu/Documents/Counties/cnty_Lyme_disease.dbf')

# Convert dbf file to a pandas data frame
df = dbf.to_dataframe()

# First, identify the maximum value for each county across the time period
max_value =[max(df.loc[:,"2005":"2014"].ix[i]) for i in range(0,len(df["2005"]))]

# Next, identify the minimum value for each county across the time period
min_value = [min(df.loc[:,"2005":"2014"].ix[i]) for i in range(0,len(df["2005"]))]

# Calculate the range of the values and add back to data frame as new column called range
df["range"] = np.array(max_value) - np.array(min_value)

# Extract the columns needed for the choropleth
extracted = df.loc[:,["County","range"]]

# Create an empty Folium map
m = folium.Map([37, -122], zoom_start=6)

# Join the extracted values to the map
m.choropleth(
    geo_str=open(counties).read(),
    data=extracted,
    columns=['County', 'range'],
    key_on='properties.County',
    fill_color='YlOrRd',
)

# Save the map as html
m.save(os.path.join("/home/ubuntu/Documents/Counties/viz/range_countymap.html"))

# Print the map to screen
m